# AOC 2025 - Day 08

## Part 1

Equipped with a new understanding of teleporter maintenance, you confidently step onto the repaired teleporter pad.

You rematerialize on an unfamiliar teleporter pad and find yourself in a vast underground space which contains a giant playground!

Across the playground, a group of Elves are working on setting up an ambitious Christmas decoration project. Through careful rigging, they have suspended a large number of small electrical junction boxes.

Their plan is to connect the junction boxes with long strings of lights. Most of the junction boxes don't provide electricity; however, when two junction boxes are connected by a string of lights, electricity can pass between those two junction boxes.

The Elves are trying to figure out **which junction boxes to connect** so that electricity can reach **every** junction box. They even have a list of all of the junction boxes' positions in 3D space (your puzzle input).

For example:

```
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
```

This list describes the position of 20 junction boxes, one per line. Each position is given as `X`,`Y`,`Z` coordinates. So, the first junction box in the list is at `X=162`, `Y=817`, `Z=812`.

To save on string lights, the Elves would like to focus on connecting pairs of junction boxes that are **as close together as possible** according to straight-line distance. In this example, the two junction boxes which are closest together are `162,817,812` and `425,690,689`.

By connecting these two junction boxes together, because electricity can flow between them, they become part of the same **circuit**. After connecting them, there is a single circuit which contains two junction boxes, and the remaining 18 junction boxes remain in their own individual circuits.

Now, the two junction boxes which are closest together but aren't already directly connected are `162,817,812` and `431,825,988`. After connecting them, since `162,817,812` is already connected to another junction box, there is now a single circuit which contains **three** junction boxes and an additional 17 circuits which contain one junction box each.

The next two junction boxes to connect are `906,360,560` and `805,96,715`. After connecting them, there is a circuit containing 3 junction boxes, a circuit containing 2 junction boxes, and 15 circuits which contain one junction box each.

The next two junction boxes are `431,825,988` and `425,690,689`. Because these two junction boxes were **already in the same circuit**, nothing happens!

This process continues for a while, and the Elves are concerned that they don't have enough extension cables for all these circuits. They would like to know how big the circuits will be.

After making the ten shortest connections, there are 11 circuits: one circuit which contains **5** junction boxes, one circuit which contains **4** junction boxes, two circuits which contain **2** junction boxes each, and seven circuits which each contain a single junction box. Multiplying together the sizes of the three largest circuits (5, 4, and one of the circuits of size 2) produces **40**.

Your list contains many junction boxes; connect together the 1000 pairs of junction boxes which are closest together. Afterward, what do you get if you multiply together the sizes of the three largest circuits?

In [18]:
import numpy as np
from scipy.spatial.distance import cdist, pdist, squareform

In [2]:
sample_input = '162,817,812\n57,618,57\n906,360,560\n592,479,940\n352,342,300\n466,668,158\n542,29,236\n431,825,988\n739,650,466\n52,470,668\n216,146,977\n819,987,18\n117,168,530\n805,96,715\n346,949,466\n970,615,88\n941,993,340\n862,61,35\n984,92,344\n425,690,689'
sample_split = sample_input.split("\n")
sample_split

['162,817,812',
 '57,618,57',
 '906,360,560',
 '592,479,940',
 '352,342,300',
 '466,668,158',
 '542,29,236',
 '431,825,988',
 '739,650,466',
 '52,470,668',
 '216,146,977',
 '819,987,18',
 '117,168,530',
 '805,96,715',
 '346,949,466',
 '970,615,88',
 '941,993,340',
 '862,61,35',
 '984,92,344',
 '425,690,689']

In [6]:
sample_arr = np.array([x.split(',') for x in sample_split], dtype=int)

In [52]:
import math

def calc_euclidean(p1, p2):
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    dz = p2[2] - p1[2]

    distance = math.sqrt(dx**2 + dy**2 + dz**2)
    return distance

In [185]:
def get_clusters(data, max_conns=1000):
    if isinstance(data, list):
        data = np.array(data, dtype=int)

    seen = set()
    uniques = []
    
    for i, p1 in enumerate(data.tolist()):
        idx_results = []
        for j, p2 in enumerate(data.tolist()):
            if i != j:
                dist = calc_euclidean(p1, p2)
                pair = (min(i, j), max(i, j))
                if pair not in seen:
                    seen.add(pair)
                    uniques.append(((i, p1), (j, p2), dist))

    uniques.sort(key=lambda x: x[2])

    graph = defaultdict(set)
    for (i, p1), (j, p2), dist in uniques[:max_conns]:
        graph[i].add(j)
        graph[j].add(i)

    visited = set()
    clusters = []

    for start_node in graph:
        if start_node not in visited:
            cluster = []
            queue = deque([start_node])
            visited.add(start_node)

            while queue:
                node = queue.popleft()
                cluster.append(node)

                for neighbor in graph[node]:
                    if neighbor not in visited:
                        visited.add(neighbor)
                        queue.append(neighbor)

            clusters.append(cluster)

    clusters.sort(key=lambda x: len(x))
    return math.prod([len(x) for x in clusters[::-1][:3]])


In [159]:
get_clusters(sample_arr.tolist(), max_conns=10)

40

In [160]:
with open("aoc2025_08_input_1.txt", "r") as f:
    puzzle = f.read().split("\n")

In [166]:
puzzle = [x.split(',') for x in puzzle]
get_clusters(puzzle)

181584

## Part 2

The Elves were right; they **definitely** don't have enough extension cables. You'll need to keep connecting junction boxes together until they're all in **one large circuit**.

Continuing the above example, the first connection which causes all of the junction boxes to form a single circuit is between the junction boxes at `216,146,977` and `117,168,530`. The Elves need to know how far those junction boxes are from the wall so they can pick the right extension cable; multiplying the X coordinates of those two junction boxes (`216` and `117`) produces **25272**.

Continue connecting the closest unconnected pairs of junction boxes together until they're all in the same circuit. **What do you get if you multiply together the X coordinates of the last two junction boxes you need to connect?**

In [181]:
puzzle_arr = np.array(puzzle, dtype=int)

In [268]:
def get_completion_pair(data):
    data = np.array(data, dtype=int)

    dist_matrix = squareform(pdist(data))

    result = -1
    pairs = []
    n = len(data)
    for i in range(n):
        for j in range(i+1, n):
            pairs.append((dist_matrix[i, j], i, j))

    pairs.sort()

    parent = list(range(n))
    num_clusters = n

    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]

    def union(x, y):
        nonlocal num_clusters
        px, py = find(x), find(y)
        if px != py:
            parent[px] = py
            num_clusters -= 1
            return True
        return False

    for dist, i, j in pairs:
        if union(i, j):
            if num_clusters == 1:
                x1 = int(data[i][0])
                x2 = int(data[j][0])
                result = x1 * x2
                print(f"Final pair: {i} and {j}")
                print(f"Coordinates: {data[i]} and {data[j]}")
                print(f"X-coordinates: {x1} * {x2} = {result}")
                break

    return result

In [269]:
get_completion_pair(sample_arr)

Final pair: 10 and 12
Coordinates: [216 146 977] and [117 168 530]
X-coordinates: 216 * 117 = 25272


25272

In [270]:
get_completion_pair(puzzle_arr)

Final pair: 362 and 954
Coordinates: [85909 27116 87709] and [98545 34309 88609]
X-coordinates: 85909 * 98545 = 8465902405


8465902405